In [1]:
import pandas as pd
import numpy as np
import os
from  mda_core.time import Time

In [2]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [3]:
receiverType =  "Novatel"
try:
    if receiverType == "Novatel":
        dataFilePath = f'./BESTXYZ.csv'

    elif receiverType == "Accord":
        dataFilePath = f'./data.csv'

except: raise FileExistsError("File not found.")

startEpoch = Time("2023-10-27T19:47:54")

In [4]:
msmt = pd.DataFrame()
msmt = pd.read_csv(dataFilePath)
msmt

,utc,P-sol status,pos type,P-X,P-Y,P-Z,P-X sigma,P-Y sigma,P-Z sigma,V-sol status,...,diff_age,sol_age,#SVs,#solnSVs,#ggL1,#solnMultiSVs,Reserved,ext sol stat,Galileo and BeiDou sig mask,GPS and GLONASS sig mask
0,1980-01-06 00:00:24,INSUFFICIENT_OBS,NONE,0.000000e+00,0.0000,0.000000e+00,0.0000,0.0000,0.0000,INSUFFICIENT_OBS,...,0.0,0.0,0,0,0,0,0,0,NaN,NaN
1,1980-01-06 00:00:25,INSUFFICIENT_OBS,NONE,0.000000e+00,0.0000,0.000000e+00,0.0000,0.0000,0.0000,INSUFFICIENT_OBS,...,0.0,0.0,0,0,0,0,0,0,NaN,NaN
2,1980-01-06 00:00:26,INSUFFICIENT_OBS,NONE,0.000000e+00,0.0000,0.000000e+00,0.0000,0.0000,0.0000,INSUFFICIENT_OBS,...,0.0,0.0,0,0,0,0,0,0,NaN,NaN
3,1980-01-06 00:00:27,INSUFFICIENT_OBS,NONE,0.000000e+00,0.0000,0.000000e+00,0.0000,0.0000,0.0000,INSUFFICIENT_OBS,...,0.0,0.0,0,0,0,0,0,0,NaN,NaN
4,1980-01-06 00:00:28,INSUFFICIENT_OBS,NONE,0.000000e+00,0.0000,0.000000e+00,0.0000,0.0000,0.0000,INSUFFICIENT_OBS,...,0.0,0.0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9880,2023-10-27 22:33:05,SOL_COMPUTED,SINGLE,-1.076552e+06,917043.2393,-6.794105e+06,3.3814,4.1833,12.4631,SOL_COMPUTED,...,0.0,0.0,12,12,12,0,0,2,NaN,GPS L1+GLONASS L1
9881,2023-10-27 22:33:06,SOL_COMPUTED,SINGLE,-1.069009e+06,917221.4401,-6.795276e+06,3.3845,4.2015,12.5067,SOL_COMPUTED,...,0.0,0.0,12,12,12,0,0,2,NaN,GPS L1+GLONASS L1
9882,2023-10-27 22:33:07,SOL_COMPUTED,SINGLE,-1.061464e+06,917397.4678,-6.796439e+06,3.3874,4.2189,12.5482,SOL_COMPUTED,...,0.0,0.0,12,12,12,0,0,2,NaN,GPS L1+GLONASS L1
9883,2023-10-27 22:33:08,SOL_COMPUTED,SINGLE,-1.053918e+06,917571.3526,-6.797593e+06,3.3883,4.2059,12.4939,SOL_COMPUTED,...,0.0,0.0,12,12,12,0,0,2,NaN,GPS L1+GLONASS L1


In [5]:
epoch = []

if receiverType == "Novatel":
        pos = pd.read_csv(dataFilePath)
        pos = pos.loc[(pos['utc'] >= '2023-10-27 19:47:54')]
        pos = pos.drop_duplicates(subset=['utc'])
        for i in pos['utc']:
                epoch.append(Time(i).get_absolutedate())
        pos['utc'] = epoch
        cols_to_notdrop = ['utc','P-sol status','pos type','P-X','P-Y','P-Z','V-sol status','vel type','V-X','V-Y','V-Z']
        for col in pos.columns:
                if col not in cols_to_notdrop:
                        pos = pos.drop(col,axis=1)
        pos = pos.replace(['SOL_COMPUTED','INSUFFICIENT_OBS','COV_TRACE'],[0, 1, 4]) #P-sol status & V-sol status
        pos = pos.replace(['NONE','SINGLE','DOPPLER VELOCITY'],[0, 16, 8]) # pos & vel type
        pos.rename(columns = {'utc':'UTC','P-sol status':'Psolstatus', 'pos type':'postype','V-sol status':'Vsolstatus','vel type':'veltype',
                              }, inplace = True)
                        

elif receiverType == "Accord":
        pos = pd.read_csv(dataFilePath)
        pos = pos.loc[(pos['UTC Time'] >= '2023-10-27 19:47:54')]
        pos = pos.drop_duplicates(subset=['UTC Time'])
        for i in pos['UTC Time']:
                epoch.append(Time(i).get_absolutedate())
        pos["UTC Time"] = epoch
        # pos.index.name = 'UTC'
        cols_to_notdrop = ['UTC Time','Position Validity','X','Y','Z','Vx','Vy','Vz','BIST Velocity validity']
        for col in pos.columns:
                if col not in cols_to_notdrop:
                        pos = pos.drop(col,axis=1)   
        pos.rename(columns = {'UTC Time':'UTC','Position Validity':'PositionValidity',
                              'BIST Velocity validity':'BISTVelocityValidity'}, inplace = True)
pos

,UTC,Psolstatus,postype,P-X,P-Y,P-Z,Vsolstatus,veltype,V-X,V-Y,V-Z
505,2023-10-27T19:56:50.000Z,1,0,0.000000e+00,0.0000,0.000000e+00,1,0,0.0000,0.0000,0.0000
506,2023-10-27T19:56:51.000Z,1,0,0.000000e+00,0.0000,0.000000e+00,1,0,0.0000,0.0000,0.0000
507,2023-10-27T19:56:52.000Z,1,0,0.000000e+00,0.0000,0.000000e+00,1,0,0.0000,0.0000,0.0000
508,2023-10-27T19:56:53.000Z,1,0,0.000000e+00,0.0000,0.000000e+00,1,0,0.0000,0.0000,0.0000
509,2023-10-27T19:56:54.000Z,1,0,0.000000e+00,0.0000,0.000000e+00,1,0,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
9880,2023-10-27T22:33:05.000Z,0,16,-1.076552e+06,917043.2393,-6.794105e+06,0,16,7542.5153,180.0021,-1177.5548
9881,2023-10-27T22:33:06.000Z,0,16,-1.069009e+06,917221.4401,-6.795276e+06,0,16,7543.8145,177.8223,-1169.4532
9882,2023-10-27T22:33:07.000Z,0,16,-1.061464e+06,917397.4678,-6.796439e+06,0,16,7545.1055,175.6420,-1161.3853
9883,2023-10-27T22:33:08.000Z,0,16,-1.053918e+06,917571.3526,-6.797593e+06,0,16,7546.3953,173.5018,-1153.4026


In [6]:
if receiverType == 'Novatel':
    pos.to_csv('GPSMain_OnOrbit_MsmtData.csv',index=False)
elif receiverType == 'Accord':
    pos.to_csv('GPSRedundant_OnOrbit_MsmtData.csv',index=False)